<a href="https://colab.research.google.com/github/badaniyahi/Entregable-3/blob/main/Copia_de_Trabajo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3

# Función para crear la base de datos y la tabla

In [ ]:
def create_database():
    conn = sqlite3.connect('articles.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS Articles (
            ArticleID INTEGER PRIMARY KEY AUTOINCREMENT,
            Title TEXT,
            PublicationDate TEXT,
            Author TEXT,
            Category TEXT,
            Summary TEXT,
            FullText TEXT,
            Keywords TEXT,
            RelatedLinks TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Función para extraer datos de la página web

In [ ]:
def scrape_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    articles = []
    # Ajusta los selectores según el HTML de la página web
    for item in soup.select('article'):  # Cambia 'article' por el selector correcto
        title = item.select_one('.article-title').get_text(strip=True)
        publication_date = item.select_one('.date').get_text(strip=True)
        author = item.select_one('.author').get_text(strip=True)
        category = item.select_one('.category').get_text(strip=True)
        summary = item.select_one('.summary').get_text(strip=True)
        full_text = item.select_one('.full-text').get_text(strip=True)
        keywords = ', '.join(k.get_text(strip=True) for k in item.select('.keywords span'))
        related_links = ', '.join(link['href'] for link in item.select('.related-links a'))

        articles.append((title, publication_date, author, category, summary, full_text, keywords, related_links))

    return articles

# Función para insertar datos en la base de datos

In [ ]:
def insert_data(articles):
    conn = sqlite3.connect('articles.db')
    c = conn.cursor()
    c.executemany('''
        INSERT INTO Articles (Title, PublicationDate, Author, Category, Summary, FullText, Keywords, RelatedLinks)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', articles)
    conn.commit()
    conn.close()

# Función principal

In [ ]:
def main():
    url = 'https://www.digitalrefining.com/articles?categoryId=25'  # Cambia esta URL por la de la página que deseas extraer
    create_database()
    articles = scrape_data(url)
    insert_data(articles)
    print("Datos extraídos e insertados en la base de datos.")

if __name__ == '__main__':
    main()

Datos extraídos e insertados en la base de datos.


 # Conversión de extensión db a csv

In [ ]:
import sqlite3
import csv

In [ ]:
def export_db_to_csv(db_filename, csv_filename):
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    # Ejecutar una consulta para seleccionar todos los datos de la tabla
    cursor.execute("SELECT * FROM Articles")

    # Obtener los nombres de las columnas
    column_names = [description[0] for description in cursor.description]

    # Obtener los datos de la consulta
    rows = cursor.fetchall()

    # Escribir los datos en un archivo CSV
    with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)

        # Escribir los nombres de las columnas
        csvwriter.writerow(column_names)

        # Escribir las filas de datos
        csvwriter.writerows(rows)

    # Cerrar la conexión a la base de datos
    conn.close()
    print(f"Datos exportados a {csv_filename}")


# Usar la función

In [ ]:
db_filename = 'articles.db'  # Nombre del archivo de base de datos
csv_filename = 'articles.csv'  # Nombre del archivo CSV de salida

In [ ]:
export_db_to_csv(db_filename, csv_filename)

Datos exportados a articles.csv
